# Praktikum Wirtschaftsinformatik: Exercise Sheet 2 - Recipe RAG

Group Nr.: 09

Student Names: Sana Afreen & Yordanos Girma Ayele

In this exercise, you'll learn to build your first simple RAG system. We'll use a publicly available recipe database to generate new recipes based on ingredients you already have. This project is a straightforward introduction to the core concepts of RAG and making sure that we get the core packages and programs to work.

Below are many small exercises that require a bit of research on notation. Often an important function is replaced with a "None" value. Here you should insert the correct function.

## First Step: Install necessary packages.

In [48]:
! pip install fpdf

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable


In [49]:
import os
from fpdf import FPDF

Haystack is the main package in this course. We use the version 2.18, which is the last larger update of this package, to ensure that future updates do not affect the funtionality of our code.

Datasets helps us to import a dataset to retrieve from. We could also use a local dataset instead.

The sentence transformers package is needed for our embeddings. To compare the query with the entries from the documents, we need to transform the data into vectors.

Ollama is an easy way to host LLMs locally and utilize our graphics card to speed up the generation.

Huggingface_hub helps with the datasets from huggingface.

In [50]:
! pip install haystack-ai==2.18
! pip install "datasets>=2.6.1"
! pip install "sentence-transformers>=4.1.0"
! pip install ollama-haystack
! pip install huggingface_hub[hf_xet]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable
  Using cached haystack_ai-2.18.0-py3-none-any.whl.metadata (15 kB)
Using cached haystack_ai-2.18.0-py3-none-any.whl (591 kB)
  Attempting uninstall: haystack-ai
    Found existing installation: haystack-ai 2.19.0
    Uninstalling haystack-ai-2.19.0:
      Successfully uninstalled haystack-ai-2.19.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ollama-haystack 5.3.0 requires haystack-ai>=2.19.0, but you have haystack-ai 2.18.0 which is incompatible.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable
  Using cached haystack_ai-2.19.0-py3-none-any.whl.metadata (15 kB)
Using cached haystack_ai-2.19.0-py3-none-any.whl (607 kB)
  Attempting uninstall: haystack-ai
    Found existing installation: haystack-ai 2.18.0
    Uninstalling haystack-ai-2.18.0:
      Successfully uninstalled haystack-ai-2.18.0


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable


## Second Step: Documents (3 points)

##### Find the correct code to initialize the InMemoryDocumentStore from Haystack. (1 Point)

The InMemoryDocumentStore is a very simple document store with no extra services. It is great for a simple RAG system.

In [30]:
from haystack.document_stores.in_memory import InMemoryDocumentStore

# Step 1: Initialize the InMemoryDocumentStore
document_store = InMemoryDocumentStore()

##### Fetch the recipe-data given in the course and convert it into Haystack Documents.

To retrieve data for our LLM we need data in the first place. For this case we use a dataset from Huggingface. You can also use your own recipes here.

In [31]:
from datasets import load_dataset
from haystack import Document

# Step 2: Load the recipe dataset (train split)
dataset = load_dataset("AWeirdDev/all-recipes-sm", split="train")

# Step 3: Prepare the documents (concatenate steps to form the content)
docs = [Document(content="".join(doc["steps"])) for doc in dataset]

##### Your data needs to be embedded by a document embedder to be added to your document store. Try to use "sentence-transformers/all-MiniLM-L6-v2" as your embedding model. (1 Point)

The recipes are provided as texts. To compare the user's query with our recipes, we need to use an embedding function to convert the documents into vectors.

In [32]:
from haystack.components.embedders import SentenceTransformersDocumentEmbedder

# Step 4: Initialize the SentenceTransformersDocumentEmbedder with the pre-trained model
doc_embedder = SentenceTransformersDocumentEmbedder(model="sentence-transformers/all-MiniLM-L6-v2")
doc_embedder.warm_up()

##### Now add your documents into your document store. (1 Point)

Use the document store that was initialized before and add the embedded documents.

In [33]:
embedded_docs = doc_embedder.run(documents=docs)

Batches:   0%|          | 0/63 [00:00<?, ?it/s]

In [34]:
document_store.write_documents(embedded_docs['documents'])

2000

## Third step: Main RAG-System Components (7 points)

##### First you need to initialize a text embedder for the query. Make sure that you use the same embedding model in your document embedder and your text embedder. (1 Point)

The user query has to be embedded with the same embedding function as the documents to ensure that semantical similarity between query and documents is consistently found.

In [37]:
from haystack.components.embedders import SentenceTransformersTextEmbedder

text_embedder = SentenceTransformersTextEmbedder("sentence-transformers/all-MiniLM-L6-v2")

##### Initialize a InMemoryEmbeddingRetriever which has access to your document store. (2 Points)

The retriever has access to the document store and receives the user query as an input. The top_k parameter defines how many documents are retrieved. Choose a low number for top_k like 2 or 3.

In [39]:
from haystack.components.retrievers import InMemoryEmbeddingRetriever

retriever = InMemoryEmbeddingRetriever(document_store=document_store, top_k=3)

##### Define a Template Prompt to control the placement of the user query and the retrieved prompts. An example is given below. Try changing it to receive good results. (2 Points)

A bad prompt can ruin the entire outcome. To reduce the time you need to experiment an example is provided. Try changing up words or add a more detailed description of how you want the results to be presented.

In [40]:
from haystack.components.builders import PromptBuilder

# Flexible recipe prompt template
template = """
You are a helpful and friendly recipe assistant. The user has asked: 
{{ query }}


Using any relevant information you have (from retrieved documents or general knowledge), generate a clear, step-by-step recipe for the user's request.
- Do **NOT** use any special characters, markdown, or formatting like asterisks (`**`) for section titles.
- Use clear, simple language with easy-to-follow instructions.
- Present the recipe in plain text with no special characters, markdown, or formatting.
- Include ingredients if applicable, listed in a clear, easy-to-read format.
- Make sure the instructions are numbered and concise, with each step explained simply.
- Highlight key sections like INGREDIENTS and INSTRUCTIONS by writing them in ALL CAPS (without any additional characters or symbols or bold).
- Do not ask the user for further clarification—generate the recipe with the information at hand.

Answer:
"""
# Create the PromptBuilder with required variables explicitly set
prompt_builder = PromptBuilder(template=template,required_variables=["query"])

##### Download and install the Ollama Client https://ollama.com/download. After installing Ollama use a terminal to download Llama 3.1 with the command "ollama pull llama3.1:8b". (2 Points)

Detailed description:
- Ollama helps us to run LLMs locally and using our GPU to decrease the time needed for generation.
- Ollama is a program that has to be running while you are running your code. By default it is auto-starting with your device. If you want to disable that, remember to start Ollama before you run the code.
- The command *ollama pull* downloads a model. We will mostly be working with llama3.1:8b. If you want to experiment a bit, try other models. Just make sure, that your PC can handle these models first.
- If the code we use different variables which are not self-explanatory:
    - timeout: The number of seconds before throwing a timeout error from the Ollama API.
    - num_ctx: Sets the size of the context window used to generate the next token.
    - temperature: The temperature of the model. Increasing the temperature will make the model answer more creatively. Try experimenting with the temperature to see which one fits your preferences the best.

In [41]:
from haystack_integrations.components.generators.ollama import OllamaGenerator

generator = OllamaGenerator(model="llama3.1",
                            url = "http://localhost:11434",
                            timeout = 30*60,
                            generation_kwargs={
                              "num_ctx": 4096,
                              "temperature": 0.9,
                              })

## Assemble and test the pipeline. (10 Points)

##### To put all your components together, connect your text embedder, retriever, prompt builder and generator into a pipeline. (2 Points)

Use the haystack website to research the inputs and outputs needed for your components to put everything together. Example pipelines are also given.

In [42]:
from haystack import Pipeline

basic_rag_pipeline = Pipeline()

In [43]:
# Creating RAG pipeline
basic_rag_pipeline.add_component("text_embedder", text_embedder)
basic_rag_pipeline.add_component("retriever", retriever)
basic_rag_pipeline.add_component("prompt_builder", prompt_builder)
basic_rag_pipeline.add_component("llm", generator)

# Connecting outputs to inputs
basic_rag_pipeline.connect("text_embedder.embedding", "retriever.query_embedding")  # sending query embedding to retriever
basic_rag_pipeline.connect("prompt_builder.prompt", "llm.prompt")                   # sending final prompt to LLM

🚅 Components
  - text_embedder: SentenceTransformersTextEmbedder
  - retriever: InMemoryEmbeddingRetriever
  - prompt_builder: PromptBuilder
  - llm: OllamaGenerator
🛤️ Connections
  - text_embedder.embedding -> retriever.query_embedding (list[float])
  - prompt_builder.prompt -> llm.prompt (str)

##### Ask a question and run your pipeline with that given question. (2 Points)

Add the variables, that are necessary to run your pipeline.

In [ ]:
# Creating Function to generate recipe for multiple queries
def generate_recipes_for_queries(queries):
    responses = []
    
    for query in queries:
        response = basic_rag_pipeline.run({"text": query, "query": query})
        llm_reply = response['llm']['replies'][0]
        responses.append({"query": query, "response": llm_reply})
    
    return responses

In [ ]:
# Defining our 6 queries
queries = [
    "How to boil an egg step by step?",
    "How to make spaghetti bolognese?",
    "What is the recipe for pancakes?",
    "How do I make a chocolate cake?",
    "How to prepare a Caesar salad?",
    "What are the steps to make a smoothie?"
]

# Generating responses for all queries
responses = generate_recipes_for_queries(queries)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Print your result as a plain text. Document your questions and the given answers. Ask at least 6 **different** questions. (6 Points)

In [ ]:
# Creating PDF output
def save_responses_to_pdf(responses, filename="recipes_output.pdf"):
    pdf = FPDF()
    pdf.set_auto_page_break(auto=True, margin=15)
    pdf.add_page()
    
    pdf.set_font("Arial", size=12)
    pdf.cell(200, 10, txt="Recipe Responses", ln=True, align='C')

    # Loop through responses and adding them to the PDF
    for i, response in enumerate(responses):
        pdf.ln(10)
        pdf.cell(200, 10, txt=f"Query {i+1}: {response['query']}", ln=True)
        pdf.ln(5)
        pdf.multi_cell(0, 10, txt=f"Response: {response['response']}")
    
    pdf.output(filename)
    print(f"PDF saved as {filename}")

# Saving the responses in PDF format
save_responses_to_pdf(responses)

PDF saved as recipes_output.pdf
